In [14]:
pip install pyperclip

In [21]:
# 네이버 블로그 아이디 크롤링 - 개정, 웹사이트 ver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import csv
import pandas as pd
# 네이버 아이디 복붙용
import pyperclip
from selenium.webdriver.common.by import By

drive = webdriver.Chrome()

# 블로그 아이디 링크를 저장하기 위한 빈 리스트
id_url_list = []
# 지정 키워드와 반복 페이지 숫자 - 원하는 키워드로 변경할 것
keyword = '윤동우'
repeatPages = 1

for i in range(0, repeatPages):
    # 페이지 이동
    web_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=sim&keyword={keyword}"
    drive.get(web_url)

    id_url = drive.find_elements(By.CLASS_NAME, 'author')

    for i in id_url:
        id_url_list.append(i.get_attribute('href'))
        drive.implicitly_wait(10)
    # time.sleep(1)
    drive.implicitly_wait(10)

# 링크의 아이디 저장용 빈 리스트
id_list = []
for i in id_url_list:
    result = i.split('/')
    id_list.append(result[3])

# # 중복된 아이디 제거
set_list = set(id_list)
# # set형식을 list로 변환
final_id_list = list(set_list)
data = pd.DataFrame(final_id_list)
# mode = a 를 추가해 기존에 있던 csv를 append, 즉 overwrite형식으로 변경
data.to_csv('naver_blogId_키워드.csv', mode='a', header=False, index=False)

drive.quit()

In [28]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
# 네이버 아이디 복붙용
import pyperclip
# 서이웃 그룹 생성용
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

from collections import defaultdict
import json


# 크롤링 옵션 생성
options = webdriver.ChromeOptions()
# 백그라운드 실행 옵션 추가
options.add_argument("headless")
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

drive = webdriver.Chrome()



# 서이추 시작
# 해당 웹사이트를 가서 네이버 로그인
url = 'https://nid.naver.com/nidlogin.login'
# 접속
drive.get(url)

drive.implicitly_wait(10)            
# 본인 아이디, 비밀번호 입력

myid = ''
mypw = ''
pyperclip.copy(myid)
drive.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.CONTROL + 'V')
pyperclip.copy(mypw)
drive.find_element(By.XPATH, '//*[@id="pw"]').send_keys(Keys.CONTROL + 'V')
# 작업 10초
drive.implicitly_wait(10)

# classes > 복수값
# id &&  >고유값
drive.find_element(By.ID, 'log.login').click()
drive.implicitly_wait(10)

# # 로그인 검증하는 기능
# try:
#     error_msg = drive.find_element_by_id('error_message').text
# except:
#     cnt = 1

# try:
#     file_path = "id_list"
#     with open(file_path, "r") as json_file:
#             id_list = json.load(json_file)
# except:
#     pass

df = pd.read_csv('naver_blogId_키워드.csv', header=None)[0]

msg = '서이추 부탁드려요~'
# 아이디 반복 추출
for id in df:
    try:
        print(id)
        blog_url = 'https://m.blog.naver.com/BuddyAddForm.naver?blogId='+ id # +'&returnUrl=https%253A%252F%252Fm.blog.naver.com%252Fhiksoo'
        drive.get(blog_url)

        # 서이추 기능이 disable일 시 다음 id로 넘김
        exceptional_text = drive.find_elements(By.CLASS_NAME, 'dsc')[0].text
        conditions = [drive.find_element_by_id('bothBuddyRadio').is_enabled(),
                       "제한된" not in exceptional_text,
                       "진행중" not in exceptional_text,
                       '하루에' not in exceptional_text]
        # 오류
        
        # True
        if False not in conditions:
            # 서이추 버튼 클릭
            # implicitly_wait을 통한 작업 소요시간 고려
            drive.find_element_by_id('bothBuddyRadio').click()
            drive.implicitly_wait(5)

            # 미리 입력된 기본값을 지운다.
            drive.find_element_by_tag_name('textarea').clear()
            drive.find_element_by_tag_name('textarea').send_keys(msg)
            drive.implicitly_wait(5)        

            # 확인 버튼 누르기
            drive.find_element_by_class_name('btn_ok').click()
            drive.implicitly_wait(5)      

        
        time.sleep(0.5)
    except Exception as e:
        print('error?', e)

osydoo
error? list index out of range
